# Zipline 取消訂單

在Zipline控制訂單的函數主要有：
    
    get_open_orders(asset = None)
    cancel_order(order)
    
get_open_orders 如果沒有加入參數，就會列出所有未成交的訂單，回傳一個字典，每個key就是asset，例如symbol('2330')，對應一個list，裡面有order_object。如果加入參數，例如 get_open_orders(symbol('2330'))，就會回傳該股票未成交的訂單，是一個list，裡面有order_object。

cancel_order 裡面的參數可以搭配 get_open_orders 用order_object，或是直接用order_id (string)。
    

Order object 格式：

Event({'id': '3096bfae27824859a4f9aba7a75fb31a', 'dt': Timestamp('2022-10-17 05:30:00+0000', tz='UTC'), 'reason': None, 'created': Timestamp('2022-10-17 05:30:00+0000', tz='UTC'), 'amount': 1500000, 'filled': 0, 'commission': 0, 'stop': None, 'limit': None, 'stop_reached': False, 'limit_reached': False, 'sid': Equity(1 [1216]), 'status': <ORDER_STATUS.OPEN: 0>})

裡面的資料，例如id，可以用object.id單獨呼叫。

## 建立環境

In [1]:
import pandas as pd
import datetime
import tejapi
import time
import os
import warnings
warnings.filterwarnings('ignore')

# tej_key-------------------------------------------
tej_key ='yourkey'
tejapi.ApiConfig.api_key = tej_key  
os.environ['TEJAPI_KEY'] = tej_key
os.environ["TEJAPI_BASE"] = "https://api.tej.com.tw"

# universe and benchmark----------------------------
idx=['TWN50']   # 'TM100' 'TWN50'
benchmark=['IR0001']

# date----------------------------------------------
# set date
start='2022-10-15'
end='2022-11-05'
os.environ['mdate'] = '20221015 20221105'      # start+' '+end #'20221011 20221223'

tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)
# calendar------------------------------------------
calendar_name='TEJ'  # US equities  XTAI

# bundle_name---------------------------------------
bundle_name = 'tquant'

from zipline.utils.calendar_utils import get_calendar
if get_calendar(calendar_name).is_session(start_dt)==False:
    start_dt=get_calendar(calendar_name).next_open(start_dt)
    
if get_calendar(calendar_name).is_session(end_dt)==False:
    end_dt=get_calendar(calendar_name).previous_close(pd.Timestamp(end_dt))
    
from zipline.api import    *

from zipline import run_algorithm  
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from zipline.utils.run_algo import (get_transaction_detail,
                    get_data_for_alphalens,
                    get_record_vars)

from zipline.sources.TEJ_Api_Data import (get_Treasury_Return,
                                          get_Benchmark_Return)


from logbook import Logger, StderrHandler, INFO
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from zipline.utils import run_algo
from zipline.data import bundles

log_handler = StderrHandler(format_string='[{record.time:%Y-%m-%d %H:%M:%S.%f}]: ' +
                            '{record.level_name}: {record.func_name}: {record.message}',
                            level=INFO)
log_handler.push_application()
log = Logger('Algorithm')

coid = "1216 2330 2327 IR0001 5844"
    
os.environ['ticker'] = coid       

!zipline ingest -b tquant

Merging daily equity files:


[2023-08-15 01:23:57.931026] INFO: zipline.data.bundles.core: Ingesting tquant.


## 範例：限制交易量、暫停交易

### zipline.api.get_open_orders(self, asset=None)
- asset (Asset) – If passed and not None, return only the open orders for the given asset instead of all open orders.

### zipline.api.cancel_order(self, order_param)
- order_param (str or Order) – The order_id or order object to cancel.



In [2]:
def initialize(context):
    context.i = 0  
    context.tickers = ['1216', '2330', '2327']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage. VolumeShareSlippage(volume_limit=0.025, price_impact=0.1))
    set_commission(commission.PerDollar(cost = commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    #PART A
    open_orders = get_open_orders()
    
    for asset in open_orders:
        for o in open_orders[asset]:
            cancel_order(o)
    #PART A
    
    print(get_datetime(), symbol('2327'), 'is_stale =', data.is_stale(symbol('2327')))

    if context.i == 0:
        order(symbol('1216'), 1500000)
    
    if context.i == 2:
        order(symbol('2327'), 1000) 

    if context.i == 3:
        order(symbol('2327'), 1000) 
     
    #PART B
    open_orders = get_open_orders()
       
    for asset in open_orders:
        if data.is_stale(asset):
            for o in open_orders[asset]:
                cancel_order(o)
    #PART B
    
    record(close=data.current(context.asset, 'close'))
    record(volume=data.current(context.asset, 'volume')*1000)
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e8
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')



performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

closing_price = tejapi.get('TWN/APIPRCD',coid=['1216'], opts={'columns':['mdate','coid','open_d','close_d']}, mdate={'gte':start,'lte':end }, paginate=True)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:24:00.913679]: INFO: handle_simulation_end: Simulated 15 trading days
first open: 2022-10-17 01:01:00+00:00
last close: 2022-11-04 05:30:00+00:00


2022-10-17 05:30:00+00:00 Equity(1 [2327]) is_stale = False
2022-10-18 05:30:00+00:00 Equity(1 [2327]) is_stale = False
2022-10-19 05:30:00+00:00 Equity(1 [2327]) is_stale = False
2022-10-20 05:30:00+00:00 Equity(1 [2327]) is_stale = True
2022-10-21 05:30:00+00:00 Equity(1 [2327]) is_stale = True
2022-10-24 05:30:00+00:00 Equity(1 [2327]) is_stale = True
2022-10-25 05:30:00+00:00 Equity(1 [2327]) is_stale = True
2022-10-26 05:30:00+00:00 Equity(1 [2327]) is_stale = True
2022-10-27 05:30:00+00:00 Equity(1 [2327]) is_stale = True
2022-10-28 05:30:00+00:00 Equity(1 [2327]) is_stale = True
2022-10-31 05:30:00+00:00 Equity(1 [2327]) is_stale = False
2022-11-01 05:30:00+00:00 Equity(1 [2327]) is_stale = False
2022-11-02 05:30:00+00:00 Equity(1 [2327]) is_stale = False
2022-11-03 05:30:00+00:00 Equity(1 [2327]) is_stale = False
2022-11-04 05:30:00+00:00 Equity(1 [2327]) is_stale = False


## 1

在第一天下單1500張統一 (1216) 股票，因為超過第二天總交易量的2.5%，第二天只成交總交易量的2.5%，剩下分批消化 (細節在Slippage篇)。如果希望達到第二天限制量就停止，加入上面PART A程式碼，在每天交易前檢查並取消還沒成交的訂單。在 'status' 可以看到訂單在買完第二天的量 'filled' 後被提前取消。 

In [3]:
performance['orders'][1]

[{'id': '0b638ca2a1ef4434b36c4f0064ad06d1',
  'dt': Timestamp('2022-10-18 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-10-17 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1500000,
  'filled': 162350,
  'commission': 15108.017567085937,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.CANCELLED: 2>}]

In [4]:
transactions

,,,amount,dt,price,order_id,commission
date,asset,symbol,,,,,
2022-10-18 00:00:00+08:00,Equity(0 [1216]),1216,162350,2022-10-18 13:30:00+08:00,65.304081,0b638ca2a1ef4434b36c4f0064ad06d1,None


## 2

股票2327在10/20到10/28暫停交易 (參考上面is_stale = True)，按照原本Zipline設計，不管是暫停前一天，或是暫停期間下單，都會在恢復交易第一天成交。但如果要加入限制機制，參考上面程式PART B，在每天下單後，檢查任何暫停交易中的股票，並取消相關的訂單。可以看到，10/19下的單子(第一個)，在10/20被取消，在10/20下的單子，在當天就被取消。

In [5]:
performance['orders'][3]

[{'id': 'fb2bf15e5c574f73ad68fe6920aac1d8',
  'dt': Timestamp('2022-10-20 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-10-19 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(1 [2327]),
  'status': <ORDER_STATUS.CANCELLED: 2>},
 {'id': 'ecf1f99999d14e668eebcd55a8392be6',
  'dt': Timestamp('2022-10-20 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-10-20 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(1 [2327]),
  'status': <ORDER_STATUS.CANCELLED: 2>}]

## 取消交易、日期、limit/stop 應用

In [6]:
def initialize(context):
    context.i = 0  
    context.tickers = ['1216', '2330', '2327']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage. VolumeShareSlippage(volume_limit=0.025, price_impact=0.1))
    set_commission(commission.PerDollar(cost = commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    
    open_orders = get_open_orders()

    #PART C
    for asset in open_orders:
        for o in open_orders[asset]:
            waiting_time = (len(get_calendar(calendar_name).sessions_in_range(o.created, get_datetime())))
            if waiting_time == 10:
                cancel_order(o)
    #PART C
    
    if context.i == 0:
        order(symbol('1216'), 1000, stop_price = 66)

    if context.i == 3:
        order(symbol('1216'), 1000, stop_price = 65.5)
        
   
   
    record(close=data.current(context.asset, 'close'))
    record(volume=data.current(context.asset, 'volume')*1000)
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e8
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')



performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

closing_price = tejapi.get('TWN/APIPRCD',coid=['1216'], opts={'columns':['mdate','coid','close_d']}, mdate={'gte':start,'lte':end }, paginate=True)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:24:01.631734]: INFO: handle_simulation_end: Simulated 15 trading days
first open: 2022-10-17 01:01:00+00:00
last close: 2022-11-04 05:30:00+00:00


## 1

在第0天下單，stop_price = 66，這單會一直在open_orders，直到股票1216在第11天 (交易日) 超過66才會買入。理論上Zipline會無限期的等stop和limit達到目標，那如果希望設下限制，如果等超過十個交易日就取消，可以參考PART C。

這段程式碼先運用了order object 裡面的.created 取得訂單創立時間，再用get_datetime()取得當天時間，再用get_calendar(calendar_name).sessions_in_range(start, end) 來算出經過了幾個交易日，並建立迴圈在每天開始檢查並取消等太久的訂單。

在第三天下的單，stop_price = 65.5，在第六天就成交，沒有等超過十天，所以沒有被PART C取消掉。

註：在訂單成交日，訂單就不會出現在open_orders裡面。

In [7]:
closing_price

,mdate,coid,close_d
None,,,
0,2022-10-17,1216,65.9
1,2022-10-18,1216,65.3
2,2022-10-19,1216,65.3
3,2022-10-20,1216,64.7
4,2022-10-21,1216,64.1
5,2022-10-24,1216,65.1
6,2022-10-25,1216,65.5
7,2022-10-26,1216,65.6
8,2022-10-27,1216,65.8


In [8]:
performance['orders'][10]

[{'id': 'a4d4db24fdb84e7d927d61aa96a91374',
  'dt': Timestamp('2022-10-31 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-10-17 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 0,
  'commission': 0,
  'stop': 66.0,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.CANCELLED: 2>}]

In [9]:
performance['orders'][6]

[{'id': 'd312b3cc4960483a9b7986e92d13ba23',
  'dt': Timestamp('2022-10-25 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-10-20 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 93.33750011778255,
  'stop': 65.5,
  'limit': None,
  'stop_reached': True,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [10]:
transactions

,,,amount,dt,price,order_id,commission
date,asset,symbol,,,,,
2022-10-25 00:00:00+08:00,Equity(0 [1216]),1216,1000,2022-10-25 13:30:00+08:00,65.5,d312b3cc4960483a9b7986e92d13ba23,None
